 This program analyzes:(1) The word frequency of covid-19 of nevada hhs part by month in 2021 (2) The attitude towards covid-19 of nevada hhs part by month in 2021

## Clean the data

In [1]:
import os
import sys
import json
from pathlib import Path

import nltk

## Load Data

In [2]:
github_file_path = str(Path(os.getcwd()).parents[1]) #sets to individual github path

sys.path.insert(1, github_file_path)

In [3]:
from LegTextScraper.states.nv import nv_preprocess

In [4]:
data = nv_preprocess("nv_hhs_m_2021.json", trim=True)

In [5]:
data['4'][1]

'Chair Nguyen: [Roll was called. Committee rules and protocol were explained.] I am going to take things out of agenda order because members are presenting in other committees right now. I will open the hearing on Assembly Bill 430. Assemblywoman Brown-May has an extreme interest in all things related to health and human services. As a last-minute appointee, she spends a lot of time watching our Committee. Assembly Bill 430: Revises provisions relating to disabilities. (BDR 39-1048) Assemblywoman Tracy Brown-May, Assembly District No. 42: I appreciate all the work this Committee has been doing. We are pleased to be here to present Assembly Bill 430. With me today are Jessica Adams, Deputy Administrator with Assembly Committee on Health and Human Services April 2, 2021 the Aging and Disability Services Division of the Department of Health and Human Services, and Ace Patrick, Chair of the Nevada Statewide Independent Living Council. We are working to raise awareness about the importance 

In [6]:
data.keys()

dict_keys(['4', '5', '2', '3'])

In [7]:
raw={}

In [8]:
for i in data.keys():
    raw[i] = json.dumps(data[i]) # convert json to string

## Processing Raw Text

### Tokenization

Break up the string into words and punctuation, and create a list of words and punctuation.

In [9]:
from nltk import word_tokenize

In [10]:
text={}
for i in raw.keys():
    text[i]= [word.lower() for word in word_tokenize(raw[i])]

In [11]:
type(text['2'])

list

### Stopwords

Stopwords are non-content words that primarily has only grammatical function

In [12]:
from nltk.corpus import stopwords

In [13]:
stopwords_en = set(stopwords.words('english')) # set checking is faster than list
print(stopwords_en)

{'d', 'off', 'doesn', 'as', 't', 'won', 'its', 'which', 'down', 'who', 'ma', 'did', "don't", 'now', 'yourselves', 'for', 'more', "you're", 'll', 'me', 'them', 'into', 'theirs', 's', 'most', 'under', 'shouldn', 'few', 'm', 'these', 'out', "aren't", 'here', 'you', 'do', 'other', "won't", "it's", 'our', 'what', 'y', 'before', 'to', 'can', 'only', 'while', 'needn', "hadn't", 'isn', 'further', 'ours', "you've", "you'd", 'aren', 'don', 'myself', 'then', 'both', 'ain', 'had', 'again', 'your', 'or', "isn't", 'wasn', 'same', "you'll", 'because', 'it', 'any', 'at', 'the', 'when', "shan't", 'where', 'shan', 'very', 'o', 'am', 're', 'his', 'are', "weren't", 'if', 'couldn', 'himself', "she's", 'by', 'why', 'through', 'until', 'being', "shouldn't", 'during', "haven't", 'him', 'has', 'haven', "that'll", "hasn't", 'was', 'he', 'we', 'nor', 'doing', 'hadn', 'having', 'that', 'and', "wouldn't", 'yours', 'she', 'such', 'their', 'from', 'there', 'this', 'will', 'on', "mightn't", 'does', 'been', 'over', 'b

Ofen, we want to remove stopwords when we want to capture only the gist of the documemt

In [14]:
type(text['2'])
text_no_stopwords={}

In [15]:
for i in text.keys():
    text_no_stopwords[i] = [word for word in text[i] if word not in stopwords_en]

In [16]:
type(text_no_stopwords)

dict

Often, we want to remove the punctuations from the documents too.

In [17]:
from string import punctuation
print('From string.punctuation:', type(punctuation), punctuation)

From string.punctuation: <class 'str'> !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [18]:
text_no_stopwords.keys()

dict_keys(['4', '5', '2', '3'])

In [19]:
text_no_stopwords_punc={}
for i in text_no_stopwords.keys():
    text_no_stopwords_punc[i] = [word for word in text_no_stopwords[i] if word not in punctuation]

In [20]:
type(text_no_stopwords_punc['2'])

list

### Lemmatization
Trying to find the root word with linguistics rules (with the use of regexes)

In [ ]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

In [ ]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 
    
def lemmatize_sent(text): 
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(text)]

In [ ]:
lemmatize_sent(text_no_stopwords_punc['2'])

In [ ]:
nltk.pos_tag(text['2'])

### Remove the line breaks

In [ ]:
text_no_stopwords_punc.keys()

In [ ]:
text_no_stopwords_punc_lb={}
for i in text_no_stopwords_punc.keys():
    text_no_stopwords_punc_lb[i] = [word for word in text_no_stopwords_punc[i] if not word.startswith('\\n')]+ [word[2:] 
                                for word in text_no_stopwords_punc if  word.startswith('\\n')]

In [ ]:
type(text_no_stopwords_punc_lb)

In [ ]:
nltk.pos_tag(text_no_stopwords_punc_lb['2'])

In [ ]:
text_no_stopwords_punc_lb.keys()

In [ ]:
text_no_stopwords_punc_lb_lemma={}
for i in text_no_stopwords_punc_lb.keys():
    text_no_stopwords_punc_lb_lemma[i]=lemmatize_sent(text_no_stopwords_punc_lb[i])

In [ ]:
nltk.pos_tag(text_no_stopwords_punc_lb_lemma['2'])

### Erase the md

In [ ]:
text_no_stopwords_punc_lb_lemma.keys()

In [ ]:
text_no_stopwords_punc_lb_lemma_md={}
for i in text_no_stopwords_punc_lb_lemma.keys():
    text_no_stopwords_punc_lb_lemma_md[i]=[word for word in text_no_stopwords_punc_lb_lemma[i] if nltk.pos_tag([word])[0][1] != 'MD' ]

In [ ]:
type(text_no_stopwords_punc_lb_lemma_md)

In [ ]:
len(text_no_stopwords_punc_lb_lemma_md['2'])

## Word frequency of covid-19 by month comparison

In [ ]:
text_no_stopwords_punc_lb_lemma_md.keys()

In [ ]:
from nltk.probability import FreqDist
textdist={}
for i in text_no_stopwords_punc_lb_lemma_md.keys():
    textdist[i] = FreqDist(text_no_stopwords_punc_lb_lemma_md[i])


In [ ]:
textdist.keys()

In [ ]:
textdistcov={}
for i in textdist.keys():
    textdistcov[i] = textdist[i].freq('covid-19')*100

In [ ]:
textdistcov

In [ ]:
b=list(textdistcov.keys())
b.sort()
b

In [ ]:
textdistcov

In [ ]:
import matplotlib.pyplot as plt
covlist = textdistcov.items()
covlist = sorted(covlist)
x1,y1 = zip(*covlist)
plt.plot(x1,y1)


## Frequency distribution of covid-19 in noun group by month 2021

In [ ]:
text_no_stopwords_punc_lb_lemma

In [ ]:
text_no_stopwords_punc_lb_lemma_onn={}
for i in text_no_stopwords_punc_lb_lemma.keys():
    text_no_stopwords_punc_lb_lemma_onn[i]=[word for word in text_no_stopwords_punc_lb_lemma[i] if nltk.pos_tag([word])[0][1] == 'NN' ]

In [ ]:
type(text_no_stopwords_punc_lb_lemma_onn)

In [ ]:
textdistn={}
for i in text_no_stopwords_punc_lb_lemma_onn.keys():
    textdistn[i] = FreqDist(text_no_stopwords_punc_lb_lemma_onn[i])


In [ ]:
textdistncov={}
for i in textdistn.keys():
    textdistncov[i] = textdistn[i].freq('covid-19')*100

In [ ]:
textdistncov

In [ ]:
textdistn['4']['covid-19']

In [ ]:
covnlist = textdistncov.items()
covnlist = sorted(covnlist)
x2,y2 = zip(*covnlist)
plt.plot(x2,y2)

# Sentiment analysis (by month)

In [ ]:
with open('nv_hhs_m_2021.json', 'r') as file:
    data= json.load(file)

In [ ]:
data.keys()

In [ ]:
sentidata={}

for i in data.keys():
    sentidata[i]=' '.join(data[i])
    
  

## Construct the 2021 hhs's sentence list by month

In [ ]:
listsen={}

In [ ]:
for i in sentidata.keys():
    listsen[i]=nltk.tokenize.sent_tokenize(sentidata[i])

### Filter the sentences which have covid-19

In [ ]:
listsen_cov={}
for i in listsen.keys():
    listsen_cov[i]=[sen for sen in listsen[i] if ' COVID-19' in sen]

In [ ]:
from textblob import TextBlob
blob={}
for i in listsen_cov.keys():
    blob[i] = TextBlob(' '.join(listsen_cov[i]))


In [ ]:
blob.keys()

In [ ]:
blob

In [ ]:
listsen_cov_pol={}
listsen_cov_sen={}
for i in blob.keys():
    polarity=1
    for j in range(len(blob[i].sentences)):
        if blob[i].sentences[j].sentiment.polarity<polarity:
            polarity=blob[i].sentences[j].sentiment.polarity
            sentence=blob[i].sentences[j]
    listsen_cov_pol[i]=polarity
    listsen_cov_sen[i]=sentence

In [ ]:
blob['2'].sentences[0].sentiment

In [ ]:
listsen_cov_pol

In [ ]:
listsen_cov_sen

## Plot the lowest polarity of covid sentenses by month

In [ ]:
covlistsen = listsen_cov_pol.items()
covlistsen = sorted(covlistsen)
x3,y3 = zip(*covlistsen)
plt.plot(x3,y3)

## Plot the highest polarity of covid sentenses by month

In [ ]:
listsen_cov_polp={}
listsen_cov_senp={}
for i in blob.keys():
    polarity=-1
    for j in range(len(blob[i].sentences)):
        if blob[i].sentences[j].sentiment.polarity>polarity:
            polarity=blob[i].sentences[j].sentiment.polarity
            sentence=blob[i].sentences[j]
    listsen_cov_polp[i]=polarity
    listsen_cov_senp[i]=sentence

In [ ]:
covlistsenp = listsen_cov_polp.items()
covlistsenp = sorted(covlistsenp)
x4,y4 = zip(*covlistsenp)
plt.plot(x4,y4)

In [ ]:
listsen_cov_polp

### Plot the polarity of covid sentenss by month

In [ ]:
covsen={}
for i in blob.keys():
    covsen[i]=blob[i].sentiment.polarity

In [ ]:
covsen

In [ ]:
covsensen = covsen.items()
covsensen = sorted(covsensen)
x5,y5 = zip(*covsensen)
plt.plot(x5,y5)